In [1]:
# code from mindnlp and huggingface transformers

In [2]:
import numpy as np
import mindspore
from mindspore import nn, ops, Tensor

# GPT-2 Masked Self-Attention

## GPT-2 Self-attention: 1- Creating queries, keys, and values

![gpt2-self-attention-3.png](https://jalammar.github.io/images/gpt2/gpt2-self-attention-3.png)

In [3]:
batch_size = 1
seq_len = 10
embed_dim = 768

x = Tensor(np.random.randn(batch_size, seq_len, embed_dim), mindspore.float32)

In [4]:
from mindnlp._legacy.functional import split
from mindnlp.models.utils.utils import Conv1D

c_attn = Conv1D(3 * embed_dim, embed_dim)
query, key, value = split(c_attn(x), embed_dim, axis=2)
query.shape, key.shape, value.shape

/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.7/site-packages/mindnlp/utils/download.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


((1, 10, 768), (1, 10, 768), (1, 10, 768))

![gpt2-self-attention-split-attention-heads-1.png](https://jalammar.github.io/images/gpt2/gpt2-self-attention-split-attention-heads-1.png)

![gpt2-self-attention-split-attention-heads-2.png](https://jalammar.github.io/images/gpt2/gpt2-self-attention-split-attention-heads-2.png)

In [5]:
def split_heads(tensor, num_heads, attn_head_size):
    """
    Splits hidden_size dim into attn_head_size and num_heads
    """
    new_shape = tensor.shape[:-1] + (num_heads, attn_head_size)
    tensor = tensor.view(new_shape)
    return ops.transpose(tensor, (0, 2, 1, 3))  # (batch, head, seq_length, head_features)

In [6]:
num_heads = 12
head_dim = embed_dim // num_heads

query = split_heads(query, num_heads, head_dim)
key = split_heads(key, num_heads, head_dim)
value = split_heads(value, num_heads, head_dim)

query.shape, key.shape, value.shape

((1, 12, 10, 64), (1, 12, 10, 64), (1, 12, 10, 64))

## GPT-2 Self-attention: 2- Scoring

![gpt2-self-attention-scoring.png](https://jalammar.github.io/images/gpt2/gpt2-self-attention-scoring.png)

![](https://jalammar.github.io/images/gpt2/gpt2-self-attention-scoring-2.png)

In [7]:
attn_weights = ops.matmul(query, key.swapaxes(-1, -2))

attn_weights.shape

(1, 12, 10, 10)

![](https://jalammar.github.io/images/gpt2/transformer-decoder-attention-mask-dataset.png)

In [8]:
max_positions = seq_len

bias = Tensor(np.tril(np.ones((max_positions, max_positions))).reshape(
              (1, 1, max_positions, max_positions)), mindspore.bool_)
bias

Tensor(shape=[1, 1, 10, 10], dtype=Bool, value=
[[[[ True, False, False ... False, False, False],
   [ True,  True, False ... False, False, False],
   [ True,  True,  True ... False, False, False],
   ...
   [ True,  True,  True ...  True, False, False],
   [ True,  True,  True ...  True,  True, False],
   [ True,  True,  True ...  True,  True,  True]]]])

![](https://jalammar.github.io/images/gpt2/queries-keys-attention-mask.png)

![](https://jalammar.github.io/images/gpt2/transformer-attention-mask.png)

In [9]:
from mindnlp._legacy.functional import where, softmax

attn_weights = attn_weights / ops.sqrt(ops.scalar_to_tensor(value.shape[-1]))
query_length, key_length = query.shape[-2], key.shape[-2]
causal_mask = bias[:, :, key_length - query_length: key_length, :key_length].bool()
mask_value = Tensor(np.finfo(np.float32).min, dtype=attn_weights.dtype)
attn_weights = where(causal_mask, attn_weights, mask_value)

In [12]:
np.finfo(np.float32).min

-3.4028235e+38

In [11]:
attn_weights[0, 0]

Tensor(shape=[10, 10], dtype=Float32, value=
[[ 1.45791307e-01, -3.40282347e+38, -3.40282347e+38 ... -3.40282347e+38, -3.40282347e+38, -3.40282347e+38],
 [ 3.03372920e-01,  1.87853992e-01, -3.40282347e+38 ... -3.40282347e+38, -3.40282347e+38, -3.40282347e+38],
 [ 2.68798053e-01, -4.71530288e-01, -7.39960253e-01 ... -3.40282347e+38, -3.40282347e+38, -3.40282347e+38],
 ...
 [-6.35852873e-01,  2.11123060e-02,  8.07071626e-02 ...  1.21709414e-01, -3.40282347e+38, -3.40282347e+38],
 [-3.82918388e-01, -1.46051317e-01, -2.59720534e-01 ...  1.63411722e-01, -8.62300470e-02, -3.40282347e+38],
 [ 1.63633123e-01, -1.29443496e-01, -3.18507515e-02 ... -1.13381080e-01,  5.03538430e-01,  1.39019817e-01]])

![](https://jalammar.github.io/images/gpt2/transformer-attention-masked-scores-softmax.png)

In [13]:
attn_weights = softmax(attn_weights, axis=-1)
attn_weights.shape

(1, 12, 10, 10)

In [14]:
attn_weights[0, 0]

Tensor(shape=[10, 10], dtype=Float32, value=
[[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00 ...  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
 [ 5.28847635e-01,  4.71152276e-01,  0.00000000e+00 ...  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
 [ 5.42997599e-01,  2.58986652e-01,  1.98015764e-01 ...  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
 ...
 [ 6.78379014e-02,  1.30854547e-01,  1.38889834e-01 ...  1.44702986e-01,  0.00000000e+00,  0.00000000e+00],
 [ 8.35228637e-02,  1.05846249e-01,  9.44733843e-02 ...  1.44235939e-01,  1.12371311e-01,  0.00000000e+00],
 [ 1.19870029e-01,  8.94188508e-02,  9.85854939e-02 ...  9.08667147e-02,  1.68395162e-01,  1.16955645e-01]])

![](https://jalammar.github.io/images/gpt2/gpt2-self-attention-multihead-sum-1.png)

In [15]:
attn_output = ops.matmul(attn_weights, value)

attn_output.shape

(1, 12, 10, 64)

## GPT-2 Self-attention: 3.5- Merge attention heads

![](https://jalammar.github.io/images/gpt2/gpt2-self-attention-merge-heads-1.png)

In [16]:
def merge_heads(tensor, num_heads, attn_head_size):
    """
    Merges attn_head_size dim and num_attn_heads dim into hidden_size
    """
    tensor = ops.transpose(tensor, (0, 2, 1, 3))
    new_shape = tensor.shape[:-2] + (num_heads * attn_head_size,)
    return tensor.view(new_shape)

In [17]:
attn_output = merge_heads(attn_output, num_heads, head_dim)

attn_output.shape

(1, 10, 768)

## GPT-2 Self-attention: 4- Projecting

![](https://jalammar.github.io/images/gpt2/gpt2-self-attention-project-1.png)

In [18]:
c_proj = Conv1D(embed_dim, embed_dim)

In [19]:
attn_output = c_proj(attn_output)
attn_output.shape

(1, 10, 768)